# Simulate image

## Import package

In [ ]:
import numpy as np
import pandas as pd
import scipy
import random
from scipy.stats import poisson,uniform
import matplotlib.pyplot as plt
from PIL import Image as im
import tensorflow as tf
from tensorflow import keras

## Setting

In [ ]:
kernel_value = np.array([3, 3])
image_size = np.array([10, 10])

## Definition 

In [ ]:
def Simulate_Image(n, img_h=10, img_w=10, pat_shape=np.ones((3,3))):
    Pat_h = pat_shape.shape[0]; Pat_w = pat_shape.shape[1] 
    W = np.random.random((n, img_h, img_w))
    Y_num = np.random.poisson(lam = 0.72, size = n)
    for i in np.arange(n) :
        if Y_num[i] > 0 :
            Y_i = np.random.choice(np.arange(img_h-Pat_h+1), Y_num[i]).astype(int)   
            Y_j = np.random.choice(np.arange(img_h-Pat_w+1), Y_num[i]).astype(int)
            for k in np.arange(Y_num[i]) :
                r_p = pat_shape * random.uniform(0,1)
                W[i, Y_i[k]:Y_i[k]+3, Y_j[k]:Y_j[k]+3] = r_p
            else :
                None
                
    categorized_list = np.empty((0, 0))
    for num in Y_num:
        if num > 0:
            categorized_list = np.append(categorized_list, 1)
        else:
            categorized_list = np.append(categorized_list, 0)

    L = {'image' : W, 'label' : categorized_list}
    return(L)

## Model training 

In [ ]:
Acc_matrix = np.zeros(100)
pattern = np.matrix([[1,0,0], [1,1,0] ,[1,1,1]])

 
model = tf.keras.Sequential([
tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(10, 10, 1)),
tf.keras.layers.MaxPooling2D((2, 2)),
# tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(10, 10, 1)),
# tf.keras.layers.MaxPooling2D((2, 2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(2)
])

model.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

train, test = Simulate_Image(1000, pat_shape = pattern), Simulate_Image(500, pat_shape = pattern)
train_images, train_labels = train['image'], train['label']
test_images, test_labels = test['image'], test['label'] 

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels),
                    verbose = 0)

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    

# HeatMap

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(1,2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:

test_labels_pos = np.where(test_labels[test_labels == 1])
test_labels_select = np.random.choice(test_labels_pos[0], 1)

model.layers[-1].activation = None
last_conv_layer_name = "conv_1"
# Generate class activation heatmap
heatmap = make_gradcam_heatmap(np.expand_dims(train_images[test_labels_select,:,:], axis=3), 
                            model, 
                            last_conv_layer_name)

# print(train_images[test_labels_select,:,:].round(2))

plt.gray()
plt.imshow(train_images[int(test_labels_select),:,:], cmap='gray', interpolation='none')
plt.xticks([])
plt.yticks([])
plt.matshow(heatmap)
plt.xticks([])
plt.yticks([])
plt.show()